In [316]:
import sqlite3

conn = sqlite3.connect('mysqlite.db')
c = conn.cursor()

#create table

c.execute('''DROP TABLE evalueserve
             ''')


#commit the changes to db
conn.commit()
#close the connection
conn.close()

In [317]:
import sqlite3

conn = sqlite3.connect('mysqlite.db')
c = conn.cursor()

#create table

c.execute('''CREATE TABLE evalueserve
             (Date date, Weekday varchar(20), AAA Float, BBB Float, CCC Float, PRIMARY KEY(Date))''')


#commit the changes to db
conn.commit()
#close the connection
conn.close()

In [318]:
conn = sqlite3.connect('mysqlite.db')


In [319]:
import pandas as pd

In [320]:
pd.read_sql(con=conn,sql=  "SELECT * FROM evalueserve")

,Date,Weekday,AAA,BBB,CCC


In [321]:
data= pd.read_excel(r"C:\Users\grove\Downloads\Time Series Index data.xlsx")

In [322]:
data.head()

,Date,Weekday,AAA,BBB,CCC
0,2011-06-20,Monday,170,597,1377
1,2011-06-21,Tuesday,124,906,968
2,2011-06-22,Wednesday,105,410,659
3,2011-06-23,Thursday,157,472,1498
4,2011-06-24,Friday,183,643,1420


In [323]:
data.to_sql("evalueserve", con=conn,if_exists='append',index=False)

In [324]:
indices_data=pd.read_sql(con=conn,sql=  "SELECT * FROM evalueserve")

In [325]:
#Weighted sum of Indices:
# I have just used this for calculation not relevant as index

indices_data['Basket']=indices_data['AAA']*0.2+indices_data['BBB']*0.35 + indices_data['CCC']*0.45

In [326]:
indices_data.head()

,Date,Weekday,AAA,BBB,CCC,Basket
0,2011-06-20 00:00:00,Monday,170.0,597.0,1377.0,862.60
1,2011-06-21 00:00:00,Tuesday,124.0,906.0,968.0,777.50
2,2011-06-22 00:00:00,Wednesday,105.0,410.0,659.0,461.05
3,2011-06-23 00:00:00,Thursday,157.0,472.0,1498.0,870.70
4,2011-06-24 00:00:00,Friday,183.0,643.0,1420.0,900.65


In [327]:
#weekends

weekends=['Saturday','Sunday']
indices_data.loc[indices_data.Weekday.isin(weekends),"is_holiday"]=1

In [328]:
#weekends counts
indices_data[indices_data.is_holiday==1].Weekday.value_counts()

Saturday    522
Sunday      522
Name: Weekday, dtype: int64

In [329]:

indices_data["Month"] = indices_data['Date'].astype(str).apply(lambda x : x[5:])

In [330]:
#Holiday
indices_data.loc[indices_data.Month.isin(["12-25", "01-01"]), "is_holiday"] =1

In [331]:

indices_data[indices_data.is_holiday == 1].Weekday.value_counts(dropna=False)

Saturday    522
Sunday      522
Name: Weekday, dtype: int64

In [332]:
import numpy as np

In [333]:
indices_data.loc[indices_data["is_holiday"] == 1, "AAA"] = np.nan
indices_data.loc[indices_data["is_holiday"] == 1, "BBB"] = np.nan
indices_data.loc[indices_data["is_holiday"] == 1, "CCC"] = np.nan

indices_data.loc[indices_data["is_holiday"] == 1, "Basket"] = np.nan

In [334]:
indices_data[["AAA", "BBB", "CCC", "Basket"]] = indices_data[["AAA", "BBB", "CCC", "Basket"]].fillna(method = "ffill")

In [335]:
indices_data.head()

,Date,Weekday,AAA,BBB,CCC,Basket,is_holiday,Month
0,2011-06-20 00:00:00,Monday,170.0,597.0,1377.0,862.60,NaN,06-20 00:00:00
1,2011-06-21 00:00:00,Tuesday,124.0,906.0,968.0,777.50,NaN,06-21 00:00:00
2,2011-06-22 00:00:00,Wednesday,105.0,410.0,659.0,461.05,NaN,06-22 00:00:00
3,2011-06-23 00:00:00,Thursday,157.0,472.0,1498.0,870.70,NaN,06-23 00:00:00
4,2011-06-24 00:00:00,Friday,183.0,643.0,1420.0,900.65,NaN,06-24 00:00:00


In [336]:
indices_data.Date = pd.to_datetime(indices_data.Date)

In [337]:
indices_data1 = indices_data[(indices_data.Date>= "2012-01-02")&(indices_data.Date<= "2020-01-19")]

In [338]:
indices_data1 = indices_data1.reset_index(drop="")

In [363]:
indices_data1.loc[0,"Basket"] = 100

In [364]:
#%change in individual indices

indices_data1['AAA_pct']=indices_data1.AAA.pct_change() * 100 
indices_data1['BBB_pct']=indices_data1.BBB.pct_change() * 100 
indices_data1['CCC_pct']=indices_data1.CCC.pct_change() * 100 

In [366]:
#setting initialvalue as 100:
indices_data1.Basket_new = indices_data1.Basket_new.fillna(100)

In [367]:
indices_data1['Weighted_sum']=1+((indices_data1['AAA_pct']*0.2+indices_data1['BBB_pct']*0.35+indices_data1['CCC_pct']*0.45)/100)

In [368]:
indices_data1

,index,Date,Weekday,AAA,BBB,CCC,Basket,is_holiday,Month,AAA_pct,BBB_pct,CCC_pct,Weighted_sum,Basket_new,Basket_ret
0,196,2012-01-02,Monday,150.0,322.0,1346.0,100.00,NaN,01-02 00:00:00,NaN,NaN,NaN,NaN,100.00,NaN
1,197,2012-01-03,Tuesday,117.0,845.0,948.0,745.75,NaN,01-03 00:00:00,-22.000000,162.422360,-29.569094,1.391417,139.14,NaN
2,198,2012-01-04,Wednesday,176.0,580.0,786.0,591.90,NaN,01-04 00:00:00,50.427350,-31.360947,-17.088608,0.914193,681.76,3.899813
3,199,2012-01-05,Thursday,179.0,775.0,1333.0,906.90,NaN,01-05 00:00:00,1.704545,33.620690,69.592875,1.434249,848.93,0.245204
4,200,2012-01-06,Friday,128.0,486.0,711.0,515.65,NaN,01-06 00:00:00,-28.491620,-37.290323,-46.661665,0.602523,546.43,-0.356331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935,3131,2020-01-15,Wednesday,159.0,865.0,581.0,596.00,NaN,01-15 00:00:00,0.000000,115.174129,-13.153961,1.343917,636.41,0.052022
2936,3132,2020-01-16,Thursday,157.0,470.0,1066.0,675.60,NaN,01-16 00:00:00,-1.257862,-45.664740,83.476764,1.213303,723.13,0.136264
2937,3133,2020-01-17,Friday,108.0,966.0,795.0,717.45,NaN,01-17 00:00:00,-31.210191,105.531915,-25.422139,1.192542,805.68,0.114157
2938,3134,2020-01-18,Saturday,108.0,966.0,795.0,717.45,1.0,01-18 00:00:00,0.000000,0.000000,0.000000,1.000000,717.45,-0.109510


In [369]:
# Calculation for index levels for each Day:


for i in indices_data1.index:
    if i in indices_data1.index:
        print(i)
        prev_basket = indices_data1.loc[i, "Basket"]
        print(prev_basket, indices_data1.loc[i+1, "Weighted_sum"])
        indices_data1.loc[i+1, "Basket_new"]=round(prev_basket *indices_data1.loc[i+1, "Weighted_sum"], 2)

0
100.0 1.3914173396214224
1
745.75 0.9141926530680182
2
591.9 1.434249443633492
3
906.9000000000001 0.6025231363706849
4
515.65 1.0
5
515.65 1.0
6
515.65 1.392509815986873
7
656.4 0.9075434655535465
8
530.35 1.8824234994386013
9
1031.6 0.8131912518474261
10
650.05 0.8426932095993613
11
577.0999999999999 1.0
12
577.0999999999999 1.0
13
577.0999999999999 1.320011593697018
14
614.45 1.6474891385410082
15
931.8 0.9179658064449132
16
892.8 0.9136352689294819
17
690.25 1.0314692201604816
18
834.0 1.0
19
834.0 1.0
20
834.0 0.8762340539800494
21
726.95 0.8458804591751876
22
509.9 2.1253598766095467
23
1022.75 1.0196211542519182
24
957.45 0.8829748929025312
25
836.8 1.0
26
836.8 1.0
27
836.8 0.6586335309581131
28
585.75 1.5105794233311527
29
783.45 0.8906036324786324
30
679.85 1.000989124585636
31
704.7 0.9367181002889137
32
770.1500000000001 1.0
33
770.1500000000001 1.0
34
770.1500000000001 1.07378688489626
35
464.05 1.2989010842542088
36
611.15 1.2105191086248281
37
659.55 1.4394023076475944

350
895.9 0.9584880240292031
351
634.45 1.4065026468801527
352
956.6000000000001 0.7224241520080235
353
647.75 1.2017659683734054
354
759.6 1.0
355
759.6 1.0
356
759.6 0.9558685581779227
357
611.0 1.4522551253139253
358
927.45 0.8908395122374851
359
715.0999999999999 0.5657791855805976
360
380.35 1.9444655154068324
361
787.7 1.0
362
787.7 1.0
363
787.7 0.7872815265206681
364
535.1 1.4397914544083534
365
773.75 0.9807498210527061
366
777.45 1.1991020055667403
367
1019.95 0.940749642294629
368
828.6500000000001 1.0
369
828.6500000000001 1.0
370
828.6500000000001 0.7404163839361138
371
524.6 1.2156170779590263
372
701.85 1.3242427867837758
373
913.45 0.8897453417315451
374
779.8 0.8523982777314077
375
559.3 1.0
376
559.3 1.0
377
559.3 1.5475582781110186
378
802.15 0.9721708612573899
379
802.75 1.2035285397004745
380
766.5 0.9413214950696676
381
759.75 0.7471816994245479
382
469.29999999999995 1.0
383
469.29999999999995 1.0
384
469.29999999999995 1.715150266730758
385
856.2 0.9620153144346

674
840.4499999999999 0.9932043871686688
675
864.05 0.9232081414040857
676
656.8 1.0
677
656.8 1.0
678
656.8 1.0289468621751026
679
693.2 0.9303169723961887
680
597.8 1.0425526240332084
681
616.6500000000001 1.0565266993913305
682
574.7 1.2728035984364232
683
728.25 1.0
684
728.25 1.0
685
728.25 0.9595225931261857
686
680.4 1.1535606434118082
687
700.05 0.8701361228088457
688
638.0 1.397942805708125
689
816.0 1.0560190648596721
690
785.0 1.0
691
785.0 1.0
692
785.0 1.027381449006203
693
759.6 0.998115141734597
694
823.6500000000001 0.9524828587328588
695
668.5 0.8534317250595745
696
621.25 1.3723154156032202
697
749.75 1.0
698
749.75 1.0
699
749.75 1.0249102619238355
700
815.5 0.9971057777563068
701
663.7 0.9457153451506086
702
635.1 0.983457428536376
703
543.55 1.3181930173784604
704
657.0 1.0
705
657.0 1.0
706
657.0 0.9406058151016299
707
583.05 0.8674161380187614
708
483.85 1.035474209217226
709
518.6500000000001 1.517011882235359
710
816.95 1.0879691954817865
711
911.5500000000001 

1038
633.3 1.437082810854152
1039
962.4499999999999 0.787180385652358
1040
752.45 1.0
1041
752.45 1.0
1042
752.45 0.7344389980627237
1043
570.05 1.4052663995876036
1044
581.05 1.6444922188657394
1045
955.8 0.9925035350262489
1046
992.25 0.8575581395348837
1047
795.55 1.0
1048
795.55 1.0
1049
795.55 0.8134890276312049
1050
616.05 1.345741546786532
1051
718.4000000000001 0.8385401740831752
1052
638.9 1.5951350015960704
1053
920.4 0.7697551286674392
1054
706.0 1.0
1055
706.0 1.0
1056
706.0 0.8188699371893691
1057
453.8 1.6265121823772888
1058
834.65 0.9820563841093984
1059
626.45 1.4958450947670292
1060
902.35 0.9080601863130662
1061
640.8499999999999 1.0
1062
640.8499999999999 1.0
1063
640.8499999999999 1.285425987296327
1064
795.25 1.1717112543234358
1065
898.0 0.8806515397045056
1066
590.0 1.4549316206028409
1067
702.85 0.8330396457402095
1068
575.0999999999999 1.0
1069
575.0999999999999 1.0
1070
575.0999999999999 1.03262189024878
1071
587.3 1.5264275944097534
1072
919.5 0.963814955191

1341
621.2 1.0
1342
621.2 1.0
1343
621.2 0.9571482796692837
1344
617.65 0.9208911375200968
1345
570.45 1.8468578801549715
1346
970.05 0.8258265864779737
1347
657.4000000000001 0.6286163091385053
1348
397.3 1.0
1349
397.3 1.0
1350
397.3 1.9139232837522262
1351
829.65 0.741833372727463
1352
561.8 1.475062281210403
1353
732.25 0.8530397896197799
1354
652.05 1.0087586390217969
1355
554.95 1.0
1356
554.95 1.0
1357
554.95 1.0880473926591447
1358
621.75 1.3565080491312203
1359
777.15 0.9372854246223513
1360
660.2 0.8658691968676144
1361
515.5 0.9999469743188881
1362
503.25 1.0
1363
503.25 1.0
1364
503.25 1.088164667790267
1365
581.7 1.6842540554831644
1366
942.6499999999999 0.774025518818964
1367
751.65 1.089998747796447
1368
828.5 0.8013884820209951
1369
564.45 1.0
1370
564.45 1.0
1371
564.45 1.1032663136353087
1372
568.35 1.2861243117566654
1373
653.5 1.1243277361142505
1374
645.3499999999999 0.8501027351862489
1375
557.9000000000001 1.6821502655332243
1376
936.55 1.0
1377
936.55 1.0
1378
9

744.35 0.9514079695404997
1640
727.9 1.0865110552072157
1641
755.95 1.326800172072417
1642
1028.65 1.0
1643
1028.65 1.0
1644
1028.65 0.85593384461832
1645
818.0 0.7245203778812856
1646
512.2 1.3898208733209207
1647
631.05 0.9293629504157683
1648
588.2 1.1478562916631856
1649
612.95 1.0
1650
612.95 1.0
1651
612.95 1.452990702819244
1652
607.05 0.9308512192068549
1653
582.4 1.3850449939847185
1654
767.65 1.2754301124315641
1655
939.75 0.9817467990424252
1656
888.6500000000001 1.0
1657
888.6500000000001 1.0
1658
888.6500000000001 0.8771378341836475
1659
801.1000000000001 1.0418075720522362
1660
859.25 0.9292616225863113
1661
629.4000000000001 1.0497241017617045
1662
637.7 1.069748152568205
1663
658.0 1.0
1664
658.0 1.0
1665
658.0 1.162538999646677
1666
822.45 0.898767611490362
1667
610.0 1.1898984699877848
1668
704.65 0.7485153157725941
1669
523.05 1.6491618212995682
1670
790.2 1.0
1671
790.2 1.0
1672
790.2 1.1254179072959563
1673
885.6000000000001 0.6022082827577668
1674
471.9 1.29723952

1931
825.15 0.7819030252851811
1932
627.15 0.7986602966318658
1933
471.7 1.0659296877249709
1934
481.55 1.1257279516656875
1935
580.0 1.2066368810724353
1936
577.4 1.0
1937
577.4 1.0
1938
577.4 0.7089604799791314
1939
418.0 1.7308320580597294
1940
801.0 0.9044024864931146
1941
650.2 1.1624241232126913
1942
806.6500000000001 0.9358725628151512
1943
760.6 1.0
1944
760.6 1.0
1945
760.6 1.0954931117756328
1946
702.85 0.9508571584047377
1947
611.5 0.9790255744347465
1948
606.8 1.0104511367288382
1949
562.1500000000001 1.326697208017798
1950
787.5 1.0
1951
787.5 1.0
1952
787.5 0.7120428888548735
1953
540.9000000000001 0.9650820990756437
1954
519.55 1.3354465520643946
1955
702.95 1.4790122044241036
1956
1036.7 0.8804177104387741
1957
916.75 1.0
1958
916.75 1.0
1959
916.75 0.8388129254029435
1960
805.0 1.4819580520168432
1961
767.8 1.1181217807252384
1962
844.6500000000001 0.9640547912867581
1963
759.25 0.668814062681466
1964
497.9 1.0
1965
497.9 1.0
1966
497.9 1.137427890895121
1967
608.9 1.2

2290
927.9000000000001 0.679725780696594
2291
515.2 1.204002515341958
2292
646.8 1.1657441296321032
2293
717.5 1.0
2294
717.5 1.0
2295
717.5 1.2285068155797854
2296
863.0 0.6489571217469114
2297
564.95 1.2616529235382308
2298
733.65 0.8141343204890413
2299
453.4 1.4876692140389578
2300
712.45 1.0
2301
712.45 1.0
2302
712.45 0.9916876265059242
2303
668.4 1.1193732056785735
2304
817.75 0.8553104466096114
2305
559.55 1.3230068786416322
2306
732.5 1.1816249647083386
2307
831.4 1.0
2308
831.4 1.0
2309
831.4 0.9113086770981508
2310
726.05 0.739162483306071
2311
492.6 1.3019818615925915
2312
594.0 0.86426649377579
2313
508.1 1.5405497470819025
2314
806.8 1.0
2315
806.8 1.0
2316
806.8 0.6948804641537789
2317
513.95 1.6202361024870882
2318
892.0 0.9026713947314667
2319
579.0 0.8709698920648072
2320
459.75 1.515105913281832
2321
738.4000000000001 1.0
2322
738.4000000000001 1.0
2323
738.4000000000001 1.288557806555803
2324
931.95 0.8529680624706237
2325
793.65 1.1407435971518043
2326
575.0 0.8150

2632
669.7 1.0241057971014493
2633
713.5500000000001 1.2675714635416826
2634
601.25 0.9755430067740045
2635
576.9 0.8274860896206846
2636
448.45000000000005 1.0
2637
448.45000000000005 1.0
2638
448.45000000000005 1.388395943000782
2639
625.8499999999999 0.9341440626883665
2640
578.1 1.3720354683636065
2641
761.15 1.0090534475258237
2642
645.8 1.0804931630911623
2643
653.8 1.0
2644
653.8 1.0
2645
653.8 1.0491283391134807
2646
667.6 1.2967791122011918
2647
749.15 0.7880707041656779
2648
512.15 1.1816962801823405
2649
599.95 0.8681026149696008
2650
513.85 1.0
2651
513.85 1.0
2652
513.85 1.0750144781786972
2653
536.5 1.2801152076281497
2654
680.95 1.1024684702592027
2655
786.45 1.1704260039817513
2656
744.8000000000001 1.139441517586416
2657
913.6 1.0
2658
913.6 1.0
2659
913.6 0.6846724075122681
2660
520.15 1.2091498227131552
2661
691.7 1.194977941587089
2662
763.55 1.0606216193567064
2663
880.0 0.9778517942629906
2664
837.35 1.0
2665
837.35 1.0
2666
837.35 1.0125731060079957
2667
790.25 0

KeyError: 2940

In [377]:
#Dropped the Basket col before :

indices_data1=indices_data1.drop(columns=['index'])
indices_data1.head()

,Date,Weekday,AAA,BBB,CCC,is_holiday,Month,AAA_pct,BBB_pct,CCC_pct,Weighted_sum,Basket_new,Basket_ret
0,2012-01-02,Monday,150.0,322.0,1346.0,NaN,01-02 00:00:00,NaN,NaN,NaN,NaN,100.00,NaN
1,2012-01-03,Tuesday,117.0,845.0,948.0,NaN,01-03 00:00:00,-22.000000,162.422360,-29.569094,1.391417,139.14,NaN
2,2012-01-04,Wednesday,176.0,580.0,786.0,NaN,01-04 00:00:00,50.427350,-31.360947,-17.088608,0.914193,681.76,3.899813
3,2012-01-05,Thursday,179.0,775.0,1333.0,NaN,01-05 00:00:00,1.704545,33.620690,69.592875,1.434249,848.93,0.245204
4,2012-01-06,Friday,128.0,486.0,711.0,NaN,01-06 00:00:00,-28.491620,-37.290323,-46.661665,0.602523,546.43,-0.356331


# <center><b> ANALYTICS </center></b>

In [384]:
indices_data1.tail()

,Date,Weekday,AAA,BBB,CCC,is_holiday,Month,AAA_pct,BBB_pct,CCC_pct,Weighted_sum,Basket_new,Basket_ret
2935,2020-01-15,Wednesday,159.0,865.0,581.0,NaN,01-15 00:00:00,0.000000,115.174129,-13.153961,1.343917,636.41,0.052022
2936,2020-01-16,Thursday,157.0,470.0,1066.0,NaN,01-16 00:00:00,-1.257862,-45.664740,83.476764,1.213303,723.13,0.136264
2937,2020-01-17,Friday,108.0,966.0,795.0,NaN,01-17 00:00:00,-31.210191,105.531915,-25.422139,1.192542,805.68,0.114157
2938,2020-01-18,Saturday,108.0,966.0,795.0,1.0,01-18 00:00:00,0.000000,0.000000,0.000000,1.000000,717.45,-0.109510
2939,2020-01-19,Sunday,108.0,966.0,795.0,1.0,01-19 00:00:00,0.000000,0.000000,0.000000,1.000000,717.45,0.000000


In [390]:
#1  year Return


#Return = ((current value/ initial value)**(365/diff between trading days)-1)*100


days=(indices_data1[indices_data1["Date"] == "2020-01-19"].Date.reset_index(drop=True)[0]-indices_data1[indices_data1["Date"] == "2019-01-19"].Date.reset_index(drop=True)[0]).days

value_initial_2020 = indices_data1[indices_data1["Date"] == "2020-01-19"]["Basket_new"].reset_index(drop = True)[0]


value_initial_2019 = indices_data1[indices_data1["Date"] == "2019-01-19"]["Basket_new"].reset_index(drop = True)[0]

ret=((value_initial_2020/value_initial_2019)**(365/days)-1)*100
ret

-1.2524946665748993

In [391]:
#Total year Return


days=(indices_data1[indices_data1["Date"] == "2020-01-19"].Date.reset_index(drop=True)[0]-indices_data1[indices_data1["Date"] == "2012-01-02"].Date.reset_index(drop=True)[0]).days

value_initial_2020 = indices_data1[indices_data1["Date"] == "2020-01-19"]["Basket_new"].reset_index(drop = True)[0]


value_initial_2012 = indices_data1[indices_data1["Date"] == "2012-01-02"]["Basket_new"].reset_index(drop = True)[0]

ret=((value_initial_2020/value_initial_2012)**(365/days)-1)*100
ret

27.72690580554693

In [392]:
#5 year Return


days=(indices_data1[indices_data1["Date"] == "2020-01-19"].Date.reset_index(drop=True)[0]-indices_data1[indices_data1["Date"] == "2015-01-19"].Date.reset_index(drop=True)[0]).days

value_initial_2020 = indices_data1[indices_data1["Date"] == "2020-01-19"]["Basket_new"].reset_index(drop = True)[0]


value_initial_2015 = indices_data1[indices_data1["Date"] == "2015-01-19"]["Basket_new"].reset_index(drop = True)[0]

ret=((value_initial_2020/value_initial_2015)**(365/days)-1)*100
ret

0.12753397803144662

In [393]:
#3 year Return


days=(indices_data1[indices_data1["Date"] == "2020-01-19"].Date.reset_index(drop=True)[0]-indices_data1[indices_data1["Date"] == "2017-01-19"].Date.reset_index(drop=True)[0]).days

value_initial_2020 = indices_data1[indices_data1["Date"] == "2020-01-19"]["Basket_new"].reset_index(drop = True)[0]


value_initial_2017 = indices_data1[indices_data1["Date"] == "2017-01-19"]["Basket_new"].reset_index(drop = True)[0]

ret=((value_initial_2020/value_initial_2017)**(365/days)-1)*100
ret

-5.4454227173924785

In [394]:
#2 year Return


days=(indices_data1[indices_data1["Date"] == "2020-01-19"].Date.reset_index(drop=True)[0]-indices_data1[indices_data1["Date"] == "2018-01-19"].Date.reset_index(drop=True)[0]).days

value_initial_2020 = indices_data1[indices_data1["Date"] == "2020-01-19"]["Basket_new"].reset_index(drop = True)[0]


value_initial_2018 = indices_data1[indices_data1["Date"] == "2018-01-19"]["Basket_new"].reset_index(drop = True)[0]

ret=((value_initial_2020/value_initial_2018)**(365/days)-1)*100
ret

9.395945510235437

In [347]:
(indices_data1[indices_data1["Date"] == "2020-01-19"].Date.reset_index(drop=True)[0]-indices_data1[indices_data1["Date"] == "2018-01-19"].Date.reset_index(drop=True)[0]).days

730

In [396]:
indices_data1['Basket_ret']=indices_data1.Basket_new.pct_change() 

In [397]:
indices_data1.head()

,Date,Weekday,AAA,BBB,CCC,is_holiday,Month,AAA_pct,BBB_pct,CCC_pct,Weighted_sum,Basket_new,Basket_ret
0,2012-01-02,Monday,150.0,322.0,1346.0,NaN,01-02 00:00:00,NaN,NaN,NaN,NaN,100.00,NaN
1,2012-01-03,Tuesday,117.0,845.0,948.0,NaN,01-03 00:00:00,-22.000000,162.422360,-29.569094,1.391417,139.14,0.391400
2,2012-01-04,Wednesday,176.0,580.0,786.0,NaN,01-04 00:00:00,50.427350,-31.360947,-17.088608,0.914193,681.76,3.899813
3,2012-01-05,Thursday,179.0,775.0,1333.0,NaN,01-05 00:00:00,1.704545,33.620690,69.592875,1.434249,848.93,0.245204
4,2012-01-06,Friday,128.0,486.0,711.0,NaN,01-06 00:00:00,-28.491620,-37.290323,-46.661665,0.602523,546.43,-0.356331


In [398]:
#daily std dev for the return on whole series
daily_std_dev=indices_data1.Basket_ret.std()

In [351]:
#Total year risk

# Risk = Daily Std. Dev for given time period * sqrt( no. of trading days)


days=(indices_data1[indices_data1["Date"] == "2020-01-19"].Date.reset_index(drop=True)[0]-indices_data1[indices_data1["Date"] == "2012-01-03"].Date.reset_index(drop=True)[0]).days


risk_initial_2020 = indices_data1[(indices_data1["Date"] >= "2012-01-03")&(indices_data1["Date"] < "2020-01-19")]["Basket_ret"]
risk_initial_2020.std()*math.sqrt(252*(days))

216.15553849839318

In [406]:
#5 year risk

risk_initial_2020 = indices_data1[(indices_data1["Date"] >= "2015-01-19")&(indices_data1["Date"] < "2020-01-19")]["Basket_ret"]
risk_initial_2020.std()*math.sqrt(252*5)

8.364130503307482

In [401]:
#3 year risk

risk_initial_2020 = indices_data1[(indices_data1["Date"] >= "2017-01-19")&(indices_data1["Date"] < "2020-01-19")]["Basket_ret"]
risk_initial_2020.std()*math.sqrt(252*3)

6.503961741213764

In [403]:
#2 year risk

risk_initial_2020 = indices_data1[(indices_data1["Date"] >= "2018-01-19")&(indices_data1["Date"] < "2020-01-19")]["Basket_ret"]
risk_initial_2020.std()*math.sqrt(252*2)

5.201597070688003

In [404]:
import math

#1 year risk

risk_initial_2020 = indices_data1[(indices_data1["Date"] >= "2019-01-19")&(indices_data1["Date"] < "2020-01-19")]["Basket_ret"]
risk_initial_2020.std()*math.sqrt(252)


3.4891674365603387

In [405]:
indices_data1.head()

,Date,Weekday,AAA,BBB,CCC,is_holiday,Month,AAA_pct,BBB_pct,CCC_pct,Weighted_sum,Basket_new,Basket_ret
0,2012-01-02,Monday,150.0,322.0,1346.0,NaN,01-02 00:00:00,NaN,NaN,NaN,NaN,100.00,NaN
1,2012-01-03,Tuesday,117.0,845.0,948.0,NaN,01-03 00:00:00,-22.000000,162.422360,-29.569094,1.391417,139.14,0.391400
2,2012-01-04,Wednesday,176.0,580.0,786.0,NaN,01-04 00:00:00,50.427350,-31.360947,-17.088608,0.914193,681.76,3.899813
3,2012-01-05,Thursday,179.0,775.0,1333.0,NaN,01-05 00:00:00,1.704545,33.620690,69.592875,1.434249,848.93,0.245204
4,2012-01-06,Friday,128.0,486.0,711.0,NaN,01-06 00:00:00,-28.491620,-37.290323,-46.661665,0.602523,546.43,-0.356331
